In [1]:
from linear_sample_3c import *
import time
from collections import namedtuple
from read_images import *#不带模糊
#from read_images_flur import *#带模糊，输出左图，右图和模糊
import os
import copy
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

In [2]:
model_params = namedtuple("parameters",
                          'data_size,'
                           'mini_batch_size,'
                           'learning_rate,'
                           'total_epoch_num,'
                           'outputdir,'
                           'height,'
                           'width,'
                           'channel,'
                          'cutTop,'
                          'cutBottom,'
                          'cutLeft,'
                          'cutRight,'
                          'cPointRow,'
                          'cPointCol')

def compute_rec_loss170(est_im, real_im, compensateI, sz_params, A, cu_A, point_meshgrid):
    """
    sz_params = size_params(batch=50,
                            height=288,
                            width=360,
                            channel=3,
                            cutTop=20,
                            cutBottom=0,
                            cutLeft=0,
                            cutRight=50)
    """
    d_height, d_width = get_tps_size(sz_params)
    # d_height = sz_params.height - sz_params.cutTop
    # d_width = sz_params.width - sz_params.cutRight
    est_clip = tf.slice(est_im, [0,  sz_params.cutTop, sz_params.cutLeft, 0], [-1, d_height, d_width, -1],
                        name='est_r_clip')
    real_clip = tf.slice(real_im, [0,  sz_params.cutTop, sz_params.cutLeft, 0], [-1, d_height, d_width, -1],
                         name='r_clip')
    
    
    
    
    real_clip_gray = tf.image.rgb_to_grayscale(real_clip)
    est_im_gray = tf.image.rgb_to_grayscale(est_clip)
    print(real_clip_gray)

    #平铺   镜面反射：左图右图均有，左图映射过来，视差图为右图，所以只能消去右图的，消去左图会导致全局向镜面反射点收敛
    ##左图的镜面反射点，应当对x小的右图点
    shape = tf.shape(real_clip_gray)
    jm = tf.fill(shape,160.0)#####如何筛选出正确的镜面反射区域,使用全部rgb都大于235来判断，在白机械臂干扰，还是有问题
    
    #shape2 = tf.shape(left_im_sum)
    #jm3 = tf.fill(shape2,680.0)

    compa2=tf.less(real_clip_gray,jm)###右图没有镜面反射的区域 所有小于650的为1 50,256,256,3
    compa3=tf.less(est_im_gray,jm)###右图没有镜面反射的区域 所有小于650的为1 50,256,256,3
    
    
    
    compa = tf.to_float(compa2&compa3)
    #compa = tf.to_float(compa3)
    
    #compa = tf.expand_dims(compa, -1)
    kernel = tf.zeros([3,3,1],dtype = tf.float32)
    compa_erosion = tf.nn.erosion2d(compa, kernel, strides = [1, 1, 1, 1], rates = [1, 1, 1, 1], padding="SAME")###compa是整个图片计算损失函数的区域
    compa_rgb = tf.tile(compa_erosion,[1,1,1,3])
    
    
    ###batch误差和初除总点数
    #loss_rec_sum = tf.reduce_sum(tf.multiply(tf.square(est_clip - real_clip - compensateI),compa_rgb))
    #compa_sum = 3*tf.reduce_sum(compa_erosion)
    #loss_rec = tf.math.divide(loss_rec_sum,compa_sum) 
    
    ###batch每张图片的像素误差的平均
    #loss_rec_sum = tf.reduce_sum(tf.multiply(tf.square(est_clip - real_clip - compensateI),compa_rgb), axis = [1,2,3])
    #print(loss_rec_sum)
    #compa_sum = tf.reduce_sum(compa_rgb,axis = [1,2,3])
    #print(compa_sum)
    #loss_rec = tf.reduce_mean(tf.divide(loss_rec_sum,compa_sum))
    
    ####batc每张图片误差和平均
    loss_rec_sum = tf.reduce_sum(tf.multiply(tf.square(est_clip - real_clip - compensateI),compa_rgb), axis = [1,2,3])
    #print(loss_rec_sum)
    loss_rec = tf.reduce_mean(loss_rec_sum)
    
    compa_sum = tf.reduce_sum(compa_rgb)###这个batch的mask总点数
    
    #####计算拉普拉斯平滑项
    A_loss = cu_A - A
    A_loss = tf.reshape(A_loss, [1,200,200,16])
    A_loss_ = tf.transpose(A_loss,[3,1,2,0])
    filter_ = np.array([[1,1,1],[1,-8,1],[1,1,1]]).reshape(3,3,1,1)###3*3,输入3通道，输出3通道
    filter_1 = tf.constant(filter_,dtype = 'float32')
    Laplace_img = tf.nn.conv2d(input = A_loss_,filter = filter_1,strides=[1,1,1,1],padding='SAME')
    print(Laplace_img)####1, 248, 270, 20
    
    loss_A_smooth = 40000*tf.reduce_sum(tf.square(Laplace_img))#+0.01*tf.reduce_sum(tf.abs(Laplace_img))##0.8
    
    
    
    
    #####计算每个点的影响是否越界，每张控制图上的16个控制点位置，当前A和初始A之间的差值的平方256
    
    #使用gather函数直接提取对应位置的值进行计算  point_meshgrid
    
    #point_all = tf.gather_nd(A_loss, point_meshgrid)
    #point_all = tf.gather_nd(tf.reshape(A, [1,240,240,16]), point_meshgrid)
    #print(point_all)
    
    #loss_point = 0.1*tf.reduce_sum(tf.square(point_all)) 
    
    return loss_rec, loss_A_smooth, compa_sum


In [3]:
def train_feature(params, feature_in, left_ims, right_ims, tps_base, tps_matrix, linear_interpolator,update_total):
    print_str = 'train_tps_Step：{:4} | Reonstruction loss is {:4}  | Asmooth loss is {:4} | Total loss is {:4}' \
                ' |  feature_var_mean is{:4}'
    # with tf.Graph().as_default(), tf.device('/gpu: 0'):
    max_step = np.int32(params.total_epoch_num)
    learning_rate_init = np.float32(params.learning_rate)

    optimize_op1 = tf.train.AdamOptimizer(learning_rate_init)##z
    
    tps_mat = tps_matrix
    feature_next = feature_in
    xymeshgrid = np.load('xymeshgrid.npy')
    
    
    with tf.variable_scope(tf.get_variable_scope()):
        tps_base = tf.constant(tps_base, dtype=tf.float32)
        left = tf.constant(left_ims[0:params.data_size], dtype=tf.float32)
        right = tf.constant(right_ims[0:params.data_size], dtype=tf.float32)
        
        
        feature_f = tf.constant(feature_in, dtype=tf.float32)
        #feature_f = tf.placeholder(tf.float32, shape=(50, 16, 1))
        tps_weight_TRUE = tf.constant(tps_matrix, dtype=tf.float32)
        tps_weight_f = tf.constant(tps_matrix, dtype=tf.float32)
        #tps_weight_f = tf.placeholder(tf.float32, shape=(57600,16))
        compensateI = tf.Variable(4.3, dtype=tf.float32, name='contr_val')###亮度补偿
        
        
        feature_in_base = tf.constant(feature_in, dtype=tf.float32)
        disp_base = tf.zeros([params.data_size, 288, 360, 1], dtype=tf.float32)
        
        
        feature_input = tf.Variable(feature_f, dtype=tf.float32, name='contr_val')
        tps_weight = tf.Variable(tps_weight_f, dtype=tf.float32)
        update1 = tf.assign(feature_input,feature_f)
        update2 = tf.assign(tps_weight,tps_weight_f)
        
        

        disp2 = decoder_forward2(feature_input, feature_in_base, tps_weight, disp_base, linear_interpolator.sz_params)
        
        right_est2 = linear_interpolator.interpolate(left, disp2)###交替训练生成右图
        #loss_rec = compute_rec_loss(right_est, right, linear_interpolator.sz_params)
        
        loss_rec2, loss_A_smooth2, compa_sum2 = compute_rec_loss170(right_est2, right, compensateI, linear_interpolator.sz_params, tps_weight_TRUE, tps_weight, xymeshgrid)

        #loss_rec, loss_wt_norm, loss_rec_bijiao, loss_rec_batch = compute_rec_loss4(right_est, right, left, disp, linear_interpolator.sz_params)##使用膨胀做部分平滑
        #loss_rec = compute_rec_loss3(right_est, right, left, linear_interpolator.sz_params)
        
        
        loss2 = tf.add(loss_rec2, loss_A_smooth2, name='Total_loss2')
        
        ###优化器设置
        train_op1 = optimize_op1.minimize(loss_rec2,var_list = feature_input)#可以指定哪些参数进行迭代
    
    config = tf.ConfigProto(allow_soft_placement=True)
    print(1)
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        
        
        print(1.1)
        start_time = time.time()
        loss_rec_temp = 0.
        print(1.2)
        
        feature_before = feature_in
        feature_val = 0.
        tps_before = tps_matrix
        tps_val = 0.
        disp_val = np.zeros([params.data_size, params.height, params.width, 1])
        est_right_val = np.zeros([params.data_size, params.height, params.width, params.channel])
        step = 0
        print(2)
        
        
        feature_base_ = np.load('groundtruth_z/z_batch_invivo_mean.npy')#16*1###分别是平均的d和z
        feature_base_ =  np.tile(feature_base_,(params.data_size, 1, 1))
        
        disp_base_val = np.load('groundtruth_z/disp_batch_invivo_mean.npy')
        disp_base_val =  np.tile(disp_base_val,(params.data_size, 1, 1, 1))
        
               
        start_time = time.time()
        loss_result = []
        ######训练z
        print("训练z")
        for step in range(0, max_step):
            _, feature_val, tps_val, loss_rec_val,loss_A_smooth_val,compa_sum2_val,loss_val, disp_val, est_right_val = sess.run(
                [train_op1, feature_input, tps_weight, loss_rec2,loss_A_smooth2,compa_sum2,loss2, disp2, right_est2],
                feed_dict={feature_f:feature_next,tps_weight_f:tps_mat,feature_in_base:feature_base_, disp_base:disp_base_val})
                
            feature_next = feature_val
            if  0 == step % 10 or step + 1 == max_step:
                print('亮度补偿', sess.run(compensateI))
                loss_result.append([copy.deepcopy(loss_rec_val)*params.data_size,copy.deepcopy(compa_sum2_val)])
                tps_var_mean = np.mean(tps_val - tps_before)
                feature_var_mean = np.mean(feature_val - feature_before)
                print(loss_rec_val*20/compa_sum2_val)
                print(print_str.format(step, loss_rec_val,loss_A_smooth_val, loss_val, feature_var_mean))#当前组的loss信息
                tps_before = tps_val
                feature_before = feature_val

                loss_rec_var = np.abs(loss_rec_val - loss_rec_temp)
                loss_rec_temp = loss_rec_val
                if step >=150:#######考虑使用loss，比较一下和重建误差的区别
                    feature_val, tps_val, loss_rec_val,loss_A_smooth_val,loss_val, disp_val, est_right_val = sess.run(
                [feature_input, tps_weight, loss_rec2,loss_A_smooth2,loss2, disp2, right_est2],
                feed_dict={feature_f:feature_next,tps_weight_f:tps_mat,feature_in_base:feature_base_, disp_base:disp_base_val})
                    feature_next = feature_val
                    loss_rec_temp = loss_rec_val
                    print(loss_rec_val)
                    break
        feature = feature_val####z
        if tps_val.any() == tps_mat.any():
            print("z结束")
        tps_mat = tps_val
        print('time spent {:8} '.format(time.time() - start_time))
        distance = np.mean(np.square(feature - feature_in))
        print('初始与终值的距离为:'+str(distance))
        
            
            
             
        
        
        return feature, tps_mat, disp_val, est_right_val, distance, loss_rec_val, loss_result
    



def train_z(params, feature_in, tps_base0, left_ims, right_ims, alpha, result_path,iii, update_total):
    with tf.Graph().as_default(), tf.device('/gpu: 0'):
        linear_interpolator = LinearInterpolator(params)  # initialize linear interpolator
        tps_matrix = tps_base0
        loss_seq = np.empty(shape=[0, 3], dtype=np.float32)
        print('--------------------FEATURE---------------------')
        feature_in, tps_matrix,  disp, est_right, distance, loss_rec_val, res_loss= train_feature(params, feature_in, left_ims, right_ims, tps_base0,
                                                                           tps_matrix, linear_interpolator,update_total)
        
        np.save(os.path.join(result_path, 'disp_batch_invivo20_'+str(iii)+'.npy'), disp)
        #np.save(os.path.join(result_path, 'est_right_invivo20'+str(iii)+'.npy'), est_right)
        #np.save(os.path.join(result_path, 'tps_trained_.npy'), tps_matrix)
        #np.save(os.path.join(result_path, 'feature_trained_'+str(iii)+'.npy'), feature_in)
        #np.save(os.path.join(result_path, 'feature_trained_invivo'+str(iii)+'.npy'), feature_in)
        #np.save(os.path.join(result_path, 'feature_trained_laststep.npy'), feature_in)
        #np.save(os.path.join(result_path, 'loss_invivo20'+str(iii)+'.npy'), loss_rec_batch_val)
        return res_loss

In [4]:
#通过83.2训练出来的结果 4*5
if __name__ == '__main__':
    params = model_params(data_size=100,
                              mini_batch_size=100,
                              learning_rate=3e-1,
                              total_epoch_num=np.int32(4000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
      
                              #cutTop=40,
                              #cutBottom=0,
                              #cutLeft=0,
                              #cutRight=90,##248 270
                          
                              cutTop=44,
                              cutBottom=44,
                              cutLeft=6,
                              cutRight=154,##248 270
                              
                              cPointRow=4,
                              cPointCol=4
                              )
    #tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    #tps_base0 = np.loadtxt(r'TPS_matrix_constant.txt').astype(np.float32)
    tps_base0 = np.load(r'groundtruth_z/tps_trained_.npy')
    #tps_base0 = np.load(r'temp_A.npy')
    #tps_base0 = np.load(r'output/temp_tps_trained_.npy')
    #ids = range(1950,2000)
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    #feature_in = np.load('groundtruth_z/disp_batch_invivo_z.npy')[0:20]
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    loss = []
    #alpha = 0.0001
    alpha = 1.0
    for i in range(0,1):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)     
        left_ims = np.array(left_ims, dtype = np.float32)
        right_ims = np.array(right_ims, dtype = np.float32)
        loss.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, alpha,'output/',i, update_total=2))
        
    #np.save('output/'+ 'TPSloss_test_.npy', loss)
        
        
        

Tensor("meshgrid/mul:0", shape=(288, 360), dtype=float32, device=/device:GPU:0) Tensor("meshgrid/mul_1:0", shape=(288, 360), dtype=float32, device=/device:GPU:0)
Tensor("Reshape:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0) Tensor("Reshape_1:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Tile:0", shape=(100, 103680), dtype=float32, device=/device:GPU:0) Tensor("Tile_1:0", shape=(100, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Reshape_2:0", shape=(10368000,), dtype=float32, device=/device:GPU:0) Tensor("Reshape_3:0", shape=(10368000,), dtype=float32, device=/device:GPU:0)
--------------------FEATURE---------------------
Tensor("sub_2:0", shape=(100, 16, 1), dtype=float32, device=/device:GPU:0)
Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(100, 40000, 1), dtype=float32)
Tensor("Add:0", shape=(100, 288, 360, 1), dtype=float32, device=/device:GPU:0)
Tensor("rgb_to_grayscale:0", shape=(100, 200, 200, 1), dtype=float32, device=/devi

In [ ]:
标准TPS 4-10 219 209 257 235 208 274
新方案  4-10 214 207 245 229 204 259
97 99 67 66

In [4]:
import numpy as np

a = np.load(r'tps_standard_248_270_4_5_inner.npy')
b = np.loadtxt(r'TPS_matrix_constant.txt').astype(np.float32)
print(a.shape)
print(b.shape)

(66960, 20)
(83080, 20)


SyntaxError: can't assign to operator (<ipython-input-3-c6133eff9a00>, line 2)

In [2]:
##制作取点向量用于tf.gather，第三个损失函数
import tensorflow as tf
import numpy as np

def point_meshgrid(n):
    w = 240
    h = 240
    control_num_u = 4
    control_num_v = 4
    
    step_u = tf.cast(tf.floor(w / (control_num_u + 1)), tf.int32)####将一行分成n+1等分
    step_v = tf.cast(tf.floor(h / (control_num_v + 1)), tf.int32)

    edge_u_left = tf.cast(tf.floor(w % (control_num_u + 1) / 2), tf.int32)
    edge_v_up = tf.cast(tf.floor(h % (control_num_v + 1) / 2), tf.int32)
    edge_u_right = w % (control_num_u + 1) - edge_u_left
    edge_v_low = h % (control_num_v + 1) - edge_v_up

    range_start_u = edge_u_left + step_u
    range_start_v = edge_v_up + step_v
    range_end_u = w - edge_u_right - step_u
    range_end_v = h - edge_v_low - step_v

    control_u = tf.range(range_start_u, range_end_u + 1, delta=step_u)
    control_v = tf.range(range_start_v, range_end_v + 1, delta=step_v)

    c_u, c_v = tf.meshgrid(control_v, control_u)##扩展为栅格
    
    shape = tf.shape(c_u)
    a = tf.zeros(shape,dtype = 'int32')
    b = n*tf.ones(shape,dtype = 'int32')
    
    point_meshgrid_ = tf.stack([a,c_u,c_v,b],axis = 2)
    print(point_meshgrid_)
    point_meshgrid_ = tf.reshape(point_meshgrid_, [16,4])
    print(point_meshgrid_)

    return point_meshgrid_




res = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    

    
    
    for i in range(16):
        point_meshgrid_ = sess.run(point_meshgrid(i))
        res.append(point_meshgrid_)
    res = np.array(res)
    print(res.shape)
    res = res.reshape(-1,4)
    print(res[0])
    
    np.save('xymeshgrid.npy', res)
    
    tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    aa = tf.constant(tps_base0)
    aa = tf.reshape(aa, [1,240,240,16])
    bb = tf.gather_nd(aa,res)

    print(sess.run(aa)[0,:,:,0])
    print(sess.run(bb))
    

Tensor("stack_16:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_17:0", shape=(16, 4), dtype=int32)
Tensor("stack_17:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_18:0", shape=(16, 4), dtype=int32)
Tensor("stack_18:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_19:0", shape=(16, 4), dtype=int32)
Tensor("stack_19:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_20:0", shape=(16, 4), dtype=int32)
Tensor("stack_20:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_21:0", shape=(16, 4), dtype=int32)
Tensor("stack_21:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_22:0", shape=(16, 4), dtype=int32)
Tensor("stack_22:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_23:0", shape=(16, 4), dtype=int32)
Tensor("stack_23:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_24:0", shape=(16, 4), dtype=int32)
Tensor("stack_24:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_25:0", shape=(16, 4), dtype=int32)
Tensor("stack_25:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_26:0", shape=(1

In [5]:
import tensorflow as tf
import numpy as np



matrix = np.load('./groundtruth_z//tps_trained.npy') #读入.npy文件
tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)

a_loss = matrix - tps_base0

aa = tf.constant(a_loss,dtype = 'float32')
print(aa)

bb = tf.transpose(aa,[1,0])
bb = tf.reshape(bb, [16,240,240,1])


with tf.Session() as sess:
    
    filter_ = np.array([[0,1,0],[1,-4,1],[0,1,0]]).reshape(3,3,1,1)###3*3,输入3通道，输出3通道
    
    
    filter_1 = tf.constant(filter_,dtype = 'float32')
    Laplace_img = tf.nn.conv2d(input = bb,filter = filter_1,strides=[1,1,1,1],padding='SAME')
    
    loss_A_smooth = tf.reduce_sum(tf.square(Laplace_img))
    
    dd = sess.run(bb)
    print(matrix.max(), matrix.min())
    
    #print(sess.run(Laplace_img))
    print(sess.run(loss_A_smooth))
    






Tensor("Const_12:0", shape=(57600, 16), dtype=float32)
1.9095305 -0.49275446
0.20306881


In [1]:
import tensorflow as tf
import numpy as np
 
input1 = tf.placeholder(tf.float32,shape = (2,1))
input2 = tf.placeholder(tf.float32)
 
output = tf.multiply(input1[0], input2)
 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(output, feed_dict = {input1:[[2.0],[3.0]], input2: 3.2}))


/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

[6.4]
